In [ ]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium import webdriver
import time
import bs4
from datetime import datetime
import pymongo
import re

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


options = webdriver.FirefoxOptions()

# Going undercover (uncomment as needed):
# options.add_argument("--incognito")
# options.add_argument("--headless")

login_url = 'https://sattacademy.com/login'

start = time.time()

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
driver.get(login_url)
driver.maximize_window()

wait = WebDriverWait(driver, 10)
email_field = wait.until(EC.presence_of_element_located((By.ID, "email")))

# Find the email and password input fields and the login button
email_field = driver.find_element(By.ID, "email")
password_field = driver.find_element(By.ID, "password")
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")

# Enter your email and password
email_field.send_keys("student@klapify.com")
password_field.send_keys("kratos040471")

# Click the login button
login_button.click()

base_url = "https://sattacademy.com/job-solution/subject-question/sid=750"
driver.get(base_url)
driver.maximize_window()

time.sleep(5)

html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')

data = []


current_page = 1
pagination_uls = driver.find_elements(By.CLASS_NAME, "page-link")
pagination_url = f"{base_url}?page={current_page}"
print(len(pagination_uls))


for x in range(1,len(pagination_uls)):
    if x>0 and x<=len(pagination_uls):
        print(x)
        pagination_url = f"{base_url}?page={x}"
        html = driver.page_source
        driver.get(pagination_url)
        driver.maximize_window()

        time.sleep(5)

        html = driver.page_source
        soup = bs4.BeautifulSoup(html, 'html.parser')
        
        test_mode_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn.btn-sm.btn-danger.sa-btn-danger.test-mode")))
        driver.execute_script("arguments[0].remove();", test_mode_button)
        
        explaination_buttons = driver.find_elements(By.CLASS_NAME, "description-btn")
        j = 0

        while True:
            print(j)
            if j == len(explaination_buttons):
#                 print('here')
                break
            else:
                time.sleep(2)
                try:
                    # Try to scroll the element into view
                    driver.execute_script("arguments[0].scrollIntoView(true);", explaination_buttons[j])
                    explaination_buttons[j].click()
                    time.sleep(2)  # Wait for the div to expand
                    # Scroll the expanded div out of view by scrolling down a bit
                    driver.execute_script("window.scrollBy(0, 400);")
                except Exception as e:
#                     print(f"Error on button {j}: {str(e)}")
                    # Use JavaScript to click the element directly if scrolling fails
                    driver.execute_script("arguments[0].click();", explaination_buttons[j])
                    time.sleep(2)  # Wait for the div to expand
                    # Scroll the expanded div out of view by scrolling down a bit
                    driver.execute_script("window.scrollBy(0, 400);")

                time.sleep(2)  # Adjust sleep time if needed
                j += 1
        
        html = driver.page_source
        soup = bs4.BeautifulSoup(html, 'html.parser')
        
        question_divs = soup.find("span", {"id": "all-question-data"}).find_all("div", {"class": "card card-bordered rounded mb-5"})
        for question_div in question_divs:
            question_title = question_div.find("div", {"class": "card-title fw-bold cursor-pointer"}).find("span", {"class": "question-span"})
#             print(question_title.decode_contents())

#                     answer_divs = question_div.find("div",{"class":"reading-mode-data"}).find_all(lambda tag: tag.name == 'div' and ('col-md-6 my-2' in tag.get('class', []) or 'col-md-6 mb-2' in tag.get('class', [])))
#                     answer_divs = question_div.find("div",{"class":"reading-mode-data"}).find_all("div", {"class": ["col-md-6 my-2 ","col-md-6 mb-2"]})
#                     print('answers', len(answer_divs))
            reading_mode_data = question_div.find("div", class_="reading-mode-data")
            # Find all divs with classes 'col-md-6 my-2' or 'col-md-6 mb-2'
            answer_divs = reading_mode_data.find_all(
                lambda tag: tag.name == 'div' and 'col-md-6' in tag.get('class', []) and
                            ('my-2' in tag.get('class', []) or 'mb-2' in tag.get('class', []))
            )
            answers_arr = []
            correct_answer = ""
            for answer in answer_divs:
                answer_div = answer.find("span",{"class":"overflow-auto"})
                if answer_div:
#                     print('-->',answer_div.decode_contents())
                    answers_arr.append(answer_div.decode_contents())
                
                if answer.find("div",{"class": "sa-success fs-3 fw-bold"}):
                    correct_answer = answer_div.decode_contents()
                    
            explaination_div = question_div.find("div", {"class": "card-footer py-2 px-0 mx-0 mx-md-2 my-0 footer-div"}).find("div", {"class": "all-description"}).find("div", {"class": "px-3 pt-3 pb-1 text-dark fw-semibold fs-5"})
            explaination_content = ""
            if explaination_div:
#                 print('explainations:',explaination_div.decode_contents())
                explaination_content = explaination_div.decode_contents()
                
            tags_divs = question_div.find("div",{"class":"px-0 overflow-auto tag-div"}).find_all("a")
            tags_arr = []
            if tags_divs:
                for tags in tags_divs:
#                     print("tags_div",tags.text)
                    tags_arr.append(f"{tags['href']} || {tags.text}")
                    
            question_data = {
                "question" : question_title.decode_contents(),
                "answers" : answers_arr,
                "correct_answer" : correct_answer,
                "explaination" : explaination_content,
                "tags" : tags_arr
            }
            data.append(question_data)

 

print(data)
driver.close()
# while True:
#     print(current_page)
#     if current_page == len(pagination_uls):
#         print('here')
#         break
#     else:   
#         current_page += 1
#         time.sleep(2)
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(2)
#         pagination_uls[current_page].click()
#         time.sleep(2)
        
#             explaination_buttons = driver.find_elements(By.CLASS_NAME, "description-btn")
#             j = 0

#             while True:
#                 print(j)
#                 if j == len(explaination_buttons):
#                     print('here')
#                     break
#                 else:
#                     time.sleep(2)
#                     try:
#                         # Try to scroll the element into view
#                         driver.execute_script("arguments[0].scrollIntoView(true);", explaination_buttons[j])
#                         explaination_buttons[j].click()
#                         time.sleep(2)  # Wait for the div to expand
#                         # Scroll the expanded div out of view by scrolling down a bit
#                         driver.execute_script("window.scrollBy(0, 400);")
#                     except Exception as e:
#                         print(f"Error on button {j}: {str(e)}")
#                         # Use JavaScript to click the element directly if scrolling fails
#                         driver.execute_script("arguments[0].click();", explaination_buttons[j])
#                         time.sleep(2)  # Wait for the div to expand
#                         # Scroll the expanded div out of view by scrolling down a bit
#                         driver.execute_script("window.scrollBy(0, 400);")

#                     time.sleep(2)  # Adjust sleep time if needed
#                     j += 1

#                    question_divs = soup.find("span", {"id": "all-question-data"}).find_all("div", {"class": "card card-bordered rounded mb-5"})
#                     for question_div in question_divs:
#                         question_title = question_div.find("div", {"class": "card-title fw-bold cursor-pointer"}).find("span", {"class": "question-span"})
#                         print(question_title.get_text())

            
#             pagination_uls[current_page].click()

            
        
        

# Close the driver after finishing
# driver.quit()



====== WebDriver manager ======
Driver [C:\Users\USER\.wdm\drivers\geckodriver\win64\v0.34.0\geckodriver.exe] found in cache


10
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
8
0
1
2
3
4
5
6
7
8
9
10
11
12
9
0
[{'question': '"চর্যাপদ" কোথা থেকে আবিষ্কৃত হয়েছে? ', 'answers': ['\n<label for="kkradio_option1_908"> তিব্বত </label>\n', '\n<label for="kkradio_option2_908"> বাংলাদেশ </label>\n', '\n<label for="kkradio_option3_908">নেপাল</label>\n', '\n<label for="kkradio_option4_908"> ভারত </label>\n'], 'correct_answer': '\n<label for="kkradio_option3_908">নেপাল</label>\n', 'explaination': "\n<p>চর্যাপদ বাংলা ভাষার প্রাচীনতম পদ সংকলন তথা সাহিত্য নিদর্শন। নব্য ভারতীয় আর্যভাষারও প্রাচীনতর রচনা এটি। খ্রিষ্টীয় অষ্টম থেকে দ্বাদশ শতাব্দীর মধ্যবর্তী সময়ে রচিত এই গীতিপদাবলির রচয়িতারা ছিলেন সহজিয়া বৌদ্ধ সিদ্ধাচার্যগণ। বৌদ্ধ ধর্মের গূঢ় অর্থ সাংকেতিক রূপের আশ্রয়ে ব্যাখ্যার উদ্দেশ্যেই তাঁরা 

In [4]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium import webdriver
import time
import bs4
from datetime import datetime
import pymongo
import re

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


options = webdriver.FirefoxOptions()

# Going undercover (uncomment as needed):
# options.add_argument("--incognito")
# options.add_argument("--headless")

login_url = 'https://sattacademy.com/login'

start = time.time()

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
driver.get(login_url)
driver.maximize_window()

wait = WebDriverWait(driver, 10)
email_field = wait.until(EC.presence_of_element_located((By.ID, "email")))

# Find the email and password input fields and the login button
email_field = driver.find_element(By.ID, "email")
password_field = driver.find_element(By.ID, "password")
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")

# Enter your email and password
email_field.send_keys("student@klapify.com")
password_field.send_keys("kratos040471")

# Click the login button
login_button.click()

base_url = "https://sattacademy.com/job-solution/subject=750/chapter"
driver.get(base_url)
driver.maximize_window()

time.sleep(5)

html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')

chapter_description = soup.find("div", {"class": "fs-5 description-div mb-9"})
print(chapter_description.text.strip())

questions_link = soup.find("div", {"class": "text-center mt-5"}).find('a')['href']
print(questions_link)



====== WebDriver manager ======
Driver [C:\Users\USER\.wdm\drivers\geckodriver\win64\v0.34.0\geckodriver.exe] found in cache


চর্যাপদ বাংলা ভাষার প্রাচীনতম পদ সংকলন তথা সাহিত্য নিদর্শন। নব্য ভারতীয় আর্যভাষারও প্রাচীনতর রচনা এটি।খ্রিষ্টীয় অষ্টম থেকে দ্বাদশ শতাব্দীর মধ্যবর্তী সময়ে রচিত এই গীতিপদাবলির রচয়িতারা ছিলেন সহজিয়া বৌদ্ধ সিদ্ধাচার্যগণ।বৌদ্ধ ধর্মের গূঢ় অর্থ সাংকেতিক রূপের আশ্রয়ে ব্যাখ্যার উদ্দেশ্যেই তাঁরা পদগুলো রচনা করেছিলেন। বাংলা সাধন সংগীত শাখাটির সূত্রপাতও হয়েছিলো এই চর্যাপদ থেকেই। সে বিবেচনায় এটি একটি ধর্মগ্রন্থজাতীয় রচনা।একই সঙ্গে সমকালীন বাংলার সামাজিক ও প্রাকৃতিক চিত্রাবলি এই পদগুলোতে উজ্জ্বল। এর সাহিত্যগুণ এখনও চিত্তাকর্ষক। ১৯০৭ খ্রিস্টাব্দে মহামহোপাধ্যায় হরপ্রসাদ শাস্ত্রী, নেপালের রাজদরবারের গ্রন্থশালা থেকে চর্যার একটি খণ্ডিত পুঁথি উদ্ধার করেন। পরবর্তীতে আচার্য সুনীতিকুমার চট্টোপাধ্যায় ভাষাতাত্ত্বিক বিশ্লেষণের মাধ্যমে চর্যাপদের সঙ্গে বাংলা ভাষার অনস্বীকার্য যোগসূত্র বৈজ্ঞানিক যুক্তিসহ প্রতিষ্ঠিত করেন। চর্যাপদের প্রধান কবিগণ হলেন লুইপাদ, কাহ্নপাদ, ভুসুকুপাদ, শবরপাদ প্রমুখ।
https://sattacademy.com/job-solution/subject-question/sid=750
